# Translation Chain (v1 – June 2025 API)

A reproducible pipeline that…

1. **Pre-processes** an input text  
2. **Generates & refines** a translation in `n` feedback loops  
3. **Post-processes** the best translation against canonical references  
4. Lets you **apply edits**

In [15]:
!pip install -U "openai>=1.10.0"

In [34]:
import os, json, textwrap, re
from openai import OpenAI
from google.colab import userdata
from typing import List, Dict, Any

openai_api_key = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

MODEL_MAIN   = "gpt-4.1"
MODEL_SEARCH = "gpt-4.1"

# Input text
Paste or edit your source text below.


In [35]:
SOURCE_TEXT = """
Alle Sätze sind Resultate von Wahrheitsoperationen mit den Elementarsätzen.
Die Wahrheitsoperation ist die Art und Weise, wie aus den Elementarsätzen die Wahrheitsfunktion entsteht.
Nach dem Wesen der Wahrheitsoperation wird auf die gleiche Weise, wie aus den Elementarsätzen ihre Wahrheitsfunktion, aus Wahrheitsfunktionen eine Neue. Jede Wahrheitsoperation erzeugt aus Wahrheitsfunktionen von Elementarsätzen wieder eine Wahrheitsfunktion von Elementarsätzen, einen Satz. Das Resultat jeder Wahrheitsoperation mit den Resultaten von Wahrheitsoperationen mit Elementarsätzen ist wieder das Resultat E i n e r Wahrheitsoperation mit Elementarsätzen.
Jeder Satz ist das Resultat von Wahrheitsoperationen mit Elementarsätzen.
…
""".strip()

TARGET_LANG = "English"


---
---

# Preprocessing

#### return clean:

In [36]:
def _response_text(resp):
    """
    Pull plain text from various Responses SDK shapes.
    Adjust if your client returns a different structure.
    """
    if hasattr(resp, "output_text") and resp.output_text:
        return resp.output_text

    if hasattr(resp, "output") and resp.output:
        for block in resp.output:
            # Some SDKs: block.type == "message" -> block.content (list)
            content = getattr(block, "content", None)
            if isinstance(content, list) and content:
                for piece in content:
                    txt = getattr(piece, "text", None)
                    if txt:
                        return getattr(txt, "value", txt)
            txt = getattr(block, "text", None)
            if txt:
                return getattr(txt, "value", txt)

    # Absolute fallback: string-ify resp
    return str(resp)

_json_re = re.compile(r'(\{.*\}|\[.*\])', re.DOTALL)

def _parse_json_from_text(text):
    """
    Find the first JSON object/array in text and parse it.
    Raises ValueError if none found/parsable.
    """
    m = _json_re.search(text)
    if not m:
        raise ValueError("No JSON object/array found in model output.")
    return json.loads(m.group(1))

## Step 1: Canonical Translation Search

In [38]:
_URL_RE = re.compile(r"^https?://[^\s]+$", re.IGNORECASE)

def _validate_entries(entries: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Keep only entries that look real:
      - have a translator or translation team credited
      - have a plausible year (e.g., 1500–2100)
      - have a working-looking http(s) URL
      - have a source_title/snippet from the web tool (a sanity check that it used search)
    """
    cleaned = []
    for e in entries:
        title = (e.get("title") or "").strip()
        translator = (e.get("translator") or "").strip()
        work = (e.get("work") or "").strip()
        year = e.get("year")
        url = (e.get("source_url") or "").strip()
        source_title = (e.get("source_title") or "").strip()
        source_snippet = (e.get("source_snippet") or "").strip()

        # translator OR institutional translator required
        has_translator = bool(translator)
        # year must be int-ish and in a sane range
        try:
            year_int = int(year)
        except Exception:
            year_int = None

        if (
            title
            and (has_translator or "team" in translator.lower() or "committee" in translator.lower())
            and (year_int is not None and 1500 <= year_int <= 2100)
            and _URL_RE.match(url)
            and source_title
            and source_snippet
        ):
            cleaned.append({
                "title": title,
                "translator": translator,
                "work": work,
                "year": year_int,
                "publisher": (e.get("publisher") or "").strip(),
                "language": (e.get("language") or "English").strip(),
                "source_url": url,
                "source_title": source_title,
                "source_snippet": source_snippet,
                "notes": (e.get("notes") or "").strip()
            })
    return cleaned

def canonical_info_structured(text: str, k: int = 3) -> List[Dict[str, Any]]:
    """
    Returns a list of verified translation entries:
      [
        {
          "title": "...",               # e.g., "Analects"
          "translator": "...",          # name(s)
          "work": "...",                # collection/edition if applicable
          "year": 2003,
          "publisher": "...",
          "language": "English",
          "source_url": "https://...",
          "source_title": "Page Title from web result",
          "source_snippet": "Snippet from the web result",
          "notes": "short note"
        },
        ...
      ]
    Only items corroborated by web_search_preview should pass validation.
    """
    schema_hint = """
    Return ONLY valid JSON with this exact shape:
    {
      "translations": [
        {
          "title": "string",
          "translator": "string",
          "work": "string",
          "year": 2000,
          "publisher": "string",
          "language": "string",
          "source_url": "https://example.org/page",
          "source_title": "string",
          "source_snippet": "string",
          "notes": "string"
        }
      ]
    }
    Rules:
    - Use web_search_preview to find real, published translations (no self-made translations).
    - Include ONLY entries you can back with a working URL found via web_search_preview.
    - Pull 'source_title' and 'source_snippet' from the search result you relied on.
    - Do not include code fences or commentary—JSON only.
    - Prefer authoritative pages (publisher/university/press/library/major archive).
    - If you find more than {k}, include the most cited/authoritative ones.
    """

    prompt = f"""
    Find at least {k} published or widely cited translations of the text below.
    Focus on editions with clear translator credit and publication info.

    {schema_hint}

    TEXT:
    ---
    {text}
    ---
    """

    resp = client.responses.create(
    model=MODEL_SEARCH,
    input=prompt,
    tools=[{"type": "web_search_preview"}],
    )

    raw = _response_text(resp)
    data = _parse_json_from_text(raw)
    translations = data.get("translations", []) if isinstance(data, dict) else []

    verified = _validate_entries(translations)

    # Optional: keep a tiny trail of what got dropped for debugging
    # (you can log this instead of returning it)
    # dropped_count = max(0, len(translations) - len(verified))

    return verified

# Example: print them nicely
entries = canonical_info_structured(SOURCE_TEXT, k=3)
print(json.dumps(entries, ensure_ascii=False, indent=2))

[
  {
    "title": "Tractatus Logico-Philosophicus",
    "translator": "C. K. Ogden",
    "work": "Tractatus Logico-Philosophicus",
    "year": 1922,
    "publisher": "Kegan Paul, Trench, Trubner & Co.",
    "language": "English",
    "source_url": "https://wittgensteinproject.org/w/index.php/Tractatus_Logico-Philosophicus_%28English%29",
    "source_title": "Tractatus Logico-Philosophicus (English) - The Ludwig Wittgenstein Project",
    "source_snippet": "This digital edition is based on Project Gutenberg's Tractatus Logico-Philosophicus, by Ludwig Wittgenstein, which was produced by Jana Srna, Norbert H. Langkau, and the Online Distributed Proofreading Team at pgdp.net; the Project Gutenberg's edition is, in turn, a reproduction of Ludwig Wittgenstein. Tractatus Logico-Philosophicus, translated by F. P. Ramsey and edited by C. K. Ogden, Kegan Paul, Trench, Trubner & Co., 1922.",
    "notes": "This is the first English translation of Wittgenstein's work, edited by C. K. Ogden and tra

## Step 2: Word-by-Word Definitions, Etymologies, Connotations

In [40]:
def word_metadata(text: str) -> dict[str, dict]:
    prompt = f"""
    Provide a JSON object where each key is a distinct word
    from the text below, and each value is an object with
    fields "definition", "etymology", and "connotations".

    Return ONLY valid JSON — absolutely no extra commentary.

    TEXT:
    ---
    {text}
    ---
    """
    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )
    data = _parse_json_from_text(_response_text(resp))
    if isinstance(data, dict):
        return data
    raise ValueError("Expected a JSON object for word_metadata.")

print(word_metadata(SOURCE_TEXT))

{'Alle': {'definition': 'All; the entirety of a group.', 'etymology': "From Old High German 'allo', meaning 'all' or 'every'.", 'connotations': 'Comprehensive, inclusive.'}, 'Sätze': {'definition': 'Sentences; statements.', 'etymology': "From German 'Satz', meaning 'sentence' or 'proposition'.", 'connotations': 'Logical, formal.'}, 'sind': {'definition': "Are; third-person plural form of 'sein' (to be).", 'etymology': "From Old High German 'sint', related to the English 'is'.", 'connotations': 'Existence, being.'}, 'Resultate': {'definition': 'Results; outcomes.', 'etymology': "From Latin 'resultare', meaning 'to spring back'.", 'connotations': 'Consequence, product.'}, 'von': {'definition': 'Of; from.', 'etymology': "From Old High German 'fon'.", 'connotations': 'Relation, origin.'}, 'Wahrheitsoperationen': {'definition': 'Truth operations; logical operations concerned with truth.', 'etymology': "German 'Wahrheit' (truth) + 'Operation' (operation).", 'connotations': 'Logic, analytic, 

## Step 3: Authorial Context

In [41]:
def author_context(text: str) -> dict:
    prompt = f"""
    Give concise context about the author of the text below.
    Return ONLY a JSON object with the keys
      "author", "life_span", "era", "notes".
    If the exact year of the quoted translation is known, include it in 'notes'.

    TEXT:
    ---
    {text}
    ---
    """
    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )
    data = _parse_json_from_text(_response_text(resp))
    if isinstance(data, dict):
        return data
    raise ValueError("Expected a JSON object for author_context.")

print(author_context(SOURCE_TEXT))

{'author': 'Ludwig Wittgenstein', 'life_span': '1889-1951', 'era': 'Early 20th century', 'notes': "Austrian-British philosopher; the text is from 'Tractatus Logico-Philosophicus.' The standard English translation by C.K. Ogden was published in 1922."}


## Step 4: Phonetics and Rhythm

In [42]:
def sound_profile(text: str) -> dict:
    prompt = f"""
    Analyse the phonetic and rhythmic qualities of the text below.
    Return ONLY a JSON object with keys
      "meter", "notable_phonetics", "observations".

    TEXT:
    ---
    {text}
    ---
    """
    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )
    data = _parse_json_from_text(_response_text(resp))
    if isinstance(data, dict):
        return data
    raise ValueError("Expected a JSON object for sound_profile.")

print(sound_profile(SOURCE_TEXT))

{'meter': 'The text is written in prose, not verse, and therefore does not adhere to a strict metrical pattern. However, the sentences display a rhythmic repetition of certain syntactic structures and key terms, creating a steady, almost incantatory cadence.', 'notable_phonetics': ["Frequent use of the sibilant /s/ (e.g., 'Sätze', 'Resultate', 'Wahrheitsoperationen'), which gives the text a soft, hissing timbre.", "Recurring compounds on 'Wahrheit(s)-' and 'Elementar(satz/sätzen)', generating repeated /v/, /aː/, /t/, /s/, and /ɛ/ sounds.", 'The long German word forms and noun compounds create repeated consonant clusters (/ts/, /ʃt/, /pl/, etc.), producing a dense and deliberate enunciation.', "Rhyming and alliteration effects through repetition of initial sounds (e.g., 'Wahrheitsfunktion', 'Wahrheitsoperation', 'Wahrheitsfunktionen') reinforce sonic unity."], 'observations': ['The frequent repetition of core vocabulary gives the text a ritualistic feel and supports comprehension of abs

## Combine Preprocessing Outputs

In [43]:
canonical_translations = canonical_info(SOURCE_TEXT, k=3)
word_info             = word_metadata(SOURCE_TEXT)
author_info           = author_context(SOURCE_TEXT)
sound_info            = sound_profile(SOURCE_TEXT)

preproc_bundle = {
    "canonical_translations": canonical_translations,
    "word_info":              word_info,
    "author_info":            author_info,
    "sound_info":             sound_info,
}

print(json.dumps(preproc_bundle, ensure_ascii=False, indent=2))

{
  "canonical_translations": [
    "All propositions are results of truth-operations on elementary propositions. The truth-operation is the manner in which a truth-function arises from elementary propositions. According to the nature of truth-operations, in the same way as a truth-function arises from elementary propositions, a new truth-function arises from truth-functions. Every truth-operation generates from truth-functions of elementary propositions another truth-function of elementary propositions, a proposition. The result of every truth-operation on the results of truth-operations on elementary propositions is again the result of a single truth-operation on elementary propositions. Every proposition is the result of truth-operations on elementary propositions.",
    "All propositions are results of truth-operations on elementary propositions. A truth-operation is the way in which a truth-function is produced out of elementary propositions. It is of the essence of truth-operatio

---
---
# Processing

## Step 1: Generate Translation from Preprocessing

In [44]:
def translate_once(text: str,
                   bundle: dict,
                   prior_translation: str | None = None) -> str:
    """
    Produce a translation informed by `bundle`, optionally improving a draft.
    """
    bundle_json = json.dumps(bundle,
                             ensure_ascii=False,
                             indent=2,
                             default=str)

    system_msg = {
        "role": "developer",
        "content": textwrap.dedent(f"""
            You are an academic translator, well-versed in translational theory.

            ## Context (JSON)
            ```json
            {bundle_json}
            ```

            • Preserve meaning, style, and sonic qualities where possible.
            • If a prior draft is provided, improve it; otherwise create a new one.
            • Output **only** the translation text—no commentary.
        """)
    }

    user_msg = {
        "role": "user",
        "content": (
            prior_translation
            if prior_translation
            else f"Translate into {TARGET_LANG}:\n\n{text}"
        )
    }

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=[system_msg, user_msg]
    )
    return resp.output_text.strip()


current_translation = translate_once(SOURCE_TEXT, preproc_bundle)

## Step 2: Judge First Translation

In [45]:
def judge_translation(src: str, draft: str) -> dict:
    """
    Returns {"positives": [...], "negatives": [...]}
    """

    prompt = f"""
    Evaluate the English translation against the source.
    Return **ONLY** a JSON object with two keys:
      "positives" : string list of strengths
      "negatives" : string list of weaknesses

    <source>
    {src}
    </source>

    <translation>
    {draft}
    </translation>
    """

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )

    try:
        return json.loads(resp.output_text)
    except json.JSONDecodeError as e:
        raise ValueError("Model did not return valid JSON; consider retrying.") from e


feedback = judge_translation(SOURCE_TEXT, current_translation)


## Step 3: Revise Translation from Feedback

In [46]:
def revise_translation(src: str,
                       draft: str,
                       critique: dict) -> str:
    """
    Improve `draft` using `critique`.
    Returns the revised translation as plain text.
    """

    messages = [
        {
            "role": "developer",
            "content": textwrap.dedent(f"""
                You are revising an English literary translation.

                ## Critique
                Positives → {critique['positives']}
                Negatives → {critique['negatives']}

                • Keep all positives intact.
                • Fix every negative issue.
                • Output **only** the revised translation text—no commentary,
                  no markdown.
            """)
        },
        {
            "role": "user",
            "content": draft
        }
    ]

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=messages
    )
    return resp.output_text.strip()


current_translation = revise_translation(
    SOURCE_TEXT,
    current_translation,
    feedback
)


## Step 4: *N* iterations

In [47]:
ITERATIONS   = 3       # set number of iterations

for i in range(1, ITERATIONS):
    fb   = judge_translation(SOURCE_TEXT, current_translation)
    prev = current_translation
    current_translation = revise_translation(SOURCE_TEXT, prev, fb)
    print(f"Iteration {i} done")

Iteration 1 done
Iteration 2 done


---
---

# Post-Processing

## Step 1: Translation + Canonical Critique

In [48]:
def compare_to_canon(final: str,
                     canon: list[str]) -> str:
    """
    Returns a short critique of `final` vs. canonical translations.
    """
    prompt = textwrap.dedent(f"""
        Compare the new translation to these canonical versions. Make these observations as concise and academically technical as possible. You are a research assistant to a brilliant translator, who needs to make edits quickly, not slog through your verbose comments. Be precise, identify strenghts and weaknesses. Only return the observations, no framing language of 'certainly,' or 'of course,' etc.

        <new_translation>
        {final}
        </new_translation>

        <canonical>
        {json.dumps(canon, ensure_ascii=False, indent=2, default=str)}
        </canonical>

        • Highlight unique strengths or weaknesses of the new version.
        • Mention where it surpasses or falls short of canon.
    """)

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )
    return resp.output_text.strip()


analysis = compare_to_canon(current_translation, canonical_info)

print("### Final translation ###\n")
print(current_translation)
print("\n---\n")
print("### Comparative analysis ###\n")
print(analysis)


### Final translation ###

All propositions are the result of truth-operations on elementary propositions.
A truth-operation is the procedure by which a truth-function is generated from elementary propositions.
Just as a truth-function arises from elementary propositions by a truth-operation, so, according to the nature of the truth-operation, a new truth-function can be produced from existing truth-functions.
Each truth-operation generates, from truth-functions of elementary propositions, a new truth-function of elementary propositions—a proposition.
The result of any truth-operation applied to the results of truth-operations on elementary propositions is again the result of a s i n g l e truth-operation with elementary propositions.
…
Every proposition is again the result of truth-operations on elementary propositions.

---

### Comparative analysis ###

- "Result of truth-operations": The new translation uses "result of truth-operations" consistently, which clarifies processual deri

## Step 3: Final Edits

In [49]:
USER_EDITS = """
collapse repeated clauses
""".strip()

# e.g. “Make diction simpler in line 2”, “Keep rhyme scheme”, …

---
---

# Final Translation

In [50]:
def apply_edits(draft: str,
                canon_analysis: str = "",
                user_edits: str = "") -> str:
    """
    Improve `draft` by:
      • integrating observations from `canon_analysis`
      • applying user-supplied edits in `user_edits`
    Returns the revised translation (plain text).
    """
    if not (canon_analysis.strip() or user_edits.strip()):
        return draft

    dev_blocks = ["You are revising the translation."]

    if canon_analysis.strip():
        dev_blocks.append("## OBSERVATIONS FROM CANON COMPARISON\n"
                          + canon_analysis.strip())

    if user_edits.strip():
        dev_blocks.append("## USER EXTRA EDITS\n" + user_edits.strip())

    dev_blocks.append("Return **only** the final revised translation text.")

    messages = [
        {"role": "developer", "content": "\n\n".join(dev_blocks)},
        {"role": "user",      "content": draft}
    ]

    resp = client.responses.create(model=MODEL_MAIN, input=messages)
    return resp.output_text.strip()


final_translation = apply_edits(
    draft=current_translation,
    canon_analysis=analysis,
    user_edits=USER_EDITS
)

print("### Final-final translation ###\n")
print(final_translation)


### Final-final translation ###

All propositions are the result of truth-operations on elementary propositions.  
A truth-operation is the procedure by which a truth-function is generated from elementary propositions.  
Just as a truth-function arises from elementary propositions by a truth-operation, so, by the same kind of procedure, a new truth-function can be produced from existing truth-functions.  
Each truth-operation yields, from truth-functions of elementary propositions, a new truth-function of elementary propositions—a proposition.  
The result of applying any truth-operation to the results of other truth-operations on elementary propositions is itself the result of a single truth-operation with elementary propositions.  
…  
Every proposition is thus the result of truth-operations on elementary propositions.
